In [2]:
import sys
import os
import json
import pandas as pd



def get_malicious(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    df_list_ma = []
    df_list_ph = []
    for i in file_list:
        local_path = root_path + i
        
        filepath = os.path.join(local_path ,'dumped_result.ma.csv')
        df_list_ma.append( pd.read_csv(filepath))
        
        filepath = os.path.join(local_path ,'dumped_result.ph.csv')
        df_list_ph.append( pd.read_csv(filepath))
        
        print local_path
        
    df_ma = pd.concat(df_list_ma, axis = 0)
    df_ph = pd.concat(df_list_ph, axis = 0)
    return df_ma, df_ph

def get_total_sample(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    total_url = set()
    for i in file_list:
        local_path = root_path + i
        filepath = os.path.join(local_path ,'dumped_result.txt')
        f = open(filepath)
        recs = [json.loads(rec) for rec in f]
        for rec in recs:
            for k, v in rec.iteritems():
                rec[k] = json.loads(v)
        for rec in recs:
            total_url.add(rec['url'])
        print filepath, len(total_url)
        f.close()
    print "Total URL Count:\t%d" %len(total_url)
    return total_url

def get_malware_data(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    df_list = []
    for i in file_list:
        local_path = root_path + i
        filepath = os.path.join(local_path ,'dumped_result.ma.csv')
        df_list.append( pd.read_csv(filepath))
        print local_path
        
    df = pd.concat(df_list, axis = 0)
    df = df.loc[df["platform"] == 5003]
    df = df.loc[(df["report_type"] == 1).values + (df["report_type"] == 2).values]
    return df

def covert_and_dump(df):
    urls = list(set(df.url))
    print len(urls)
    return pd.DataFrame({"url": urls})

In [2]:
file_list = ["0301","0302","0306","0307","0311","0312","0313","0314","0315","0318","0319",
             "0320","0321","0322","0323","0325","0326","0327","0328","0330","0401","0402", 
             "0410","0411","0412","0415","0416","0417","0418","0419"]
total_url = get_total_sample(file_list)
df_ma, df_ph = get_malicious(file_list)

/home/sparrow/sparrow/data/sangfor/180301/dumped_result.txt 100000
/home/sparrow/sparrow/data/sangfor/180302/dumped_result.txt 162920
/home/sparrow/sparrow/data/sangfor/180306/dumped_result.txt 219430
/home/sparrow/sparrow/data/sangfor/180307/dumped_result.txt 267294
/home/sparrow/sparrow/data/sangfor/180311/dumped_result.txt 334200
/home/sparrow/sparrow/data/sangfor/180312/dumped_result.txt 400270
/home/sparrow/sparrow/data/sangfor/180313/dumped_result.txt 444164
/home/sparrow/sparrow/data/sangfor/180314/dumped_result.txt 486715
/home/sparrow/sparrow/data/sangfor/180315/dumped_result.txt 528351
/home/sparrow/sparrow/data/sangfor/180318/dumped_result.txt 590172
/home/sparrow/sparrow/data/sangfor/180319/dumped_result.txt 651913
/home/sparrow/sparrow/data/sangfor/180320/dumped_result.txt 691585
/home/sparrow/sparrow/data/sangfor/180321/dumped_result.txt 728972
/home/sparrow/sparrow/data/sangfor/180322/dumped_result.txt 767933
/home/sparrow/sparrow/data/sangfor/180323/dumped_result.txt 80

In [8]:
safe_url = list(set(total_url) - set(df_ma.url) - set(df_ph.url))
safe_url = [_.encode('utf-8') for _ in safe_url]
df_safe = pd.DataFrame({'url': safe_url})
print "Safe URL Count:\t%d" %len(df_safe)
df_safe.to_csv("/home/sparrow/sparrow/data/sangfor/safe_0419.csv")

Safe URL Count:	1461039


In [5]:
df = covert_and_dump(df_ma)
df_ma.to_csv("/home/sparrow/sparrow/data/sangfor/malware_all_0419.csv")

39798


In [19]:
file_list = ["0410", "0411", "0412", "0415", "0416"]
df = get_malware_data(file_list)
df.to_csv("/home/sparrow/sparrow/data/sangfor/malware0416.csv")

/home/sparrow/sparrow/data/sangfor/180410
/home/sparrow/sparrow/data/sangfor/180411
/home/sparrow/sparrow/data/sangfor/180412
/home/sparrow/sparrow/data/sangfor/180415
/home/sparrow/sparrow/data/sangfor/180416


# Get EDR data

In [12]:
def get_malware_edr_data(file_list):
    root_path = "/home/sparrow/sparrow/data/sangfor/18"
    df_list = []
    for i in file_list:
        local_path = root_path + i
        filepath = os.path.join(local_path ,'dumped_result.edr.csv')
        df_list.append( pd.read_csv(filepath))
        print local_path
        
    df = pd.concat(df_list, axis = 0)
    return df

In [13]:
file_list = ["0401", "0402", "0410", "0411", "0412", "0415", "0416", "0417", "0418", "0419"]

In [15]:
df = get_malware_edr_data(file_list)

/home/sparrow/sparrow/data/sangfor/180401
/home/sparrow/sparrow/data/sangfor/180402
/home/sparrow/sparrow/data/sangfor/180410
/home/sparrow/sparrow/data/sangfor/180411
/home/sparrow/sparrow/data/sangfor/180412
/home/sparrow/sparrow/data/sangfor/180415
/home/sparrow/sparrow/data/sangfor/180416
/home/sparrow/sparrow/data/sangfor/180417
/home/sparrow/sparrow/data/sangfor/180418
/home/sparrow/sparrow/data/sangfor/180419


# Virus Total data

In [11]:
def read_data(date):
    file_list = os.listdir("../../data/VirusTotalnew/{date}/".format(date = date))
    file_num = len(file_list)
    df_list = []
    for i in range(file_num):
        try:
            df_list.append(pd.read_csv("../../data/VirusTotalnew/{date}/vt-{date}-{i}.csv".format(date = date, i = i)))
            if i %2000 == 0:
                print("loading ../../data/VirusTotalnew/{date}/vt-{date}-{i}.csv".format(date = date, i = i))
        except Exception as e:
            logging.error("%s" %str(e))
    df = pd.concat(df_list)
    return df


def make_malicious_file(date):
    assert isinstance(date, list)
    df_list = []
    for i in date:
        df_list.append(read_data(i))
    df = pd.concat(df_list)
    return df
    
def get_data_list():
    curadspath = os.path.abspath("../../data/VirusTotalnew")
    file_list = os.listdir(curadspath)
    return [_ for _ in file_list if _.isdigit()]

In [12]:
date_list = get_data_list()
df = make_malicious_file(date_list)

loading ../../data/VirusTotalnew/20180413/vt-20180413-0.csv
loading ../../data/VirusTotalnew/20180413/vt-20180413-2000.csv
loading ../../data/VirusTotalnew/20180419/vt-20180419-0.csv
loading ../../data/VirusTotalnew/20180419/vt-20180419-2000.csv
loading ../../data/VirusTotalnew/20180419/vt-20180419-4000.csv
loading ../../data/VirusTotalnew/20180414/vt-20180414-0.csv
loading ../../data/VirusTotalnew/20180414/vt-20180414-2000.csv
loading ../../data/VirusTotalnew/20180417/vt-20180417-0.csv
loading ../../data/VirusTotalnew/20180417/vt-20180417-2000.csv
loading ../../data/VirusTotalnew/20180417/vt-20180417-4000.csv
loading ../../data/VirusTotalnew/20180424/vt-20180424-0.csv
loading ../../data/VirusTotalnew/20180424/vt-20180424-2000.csv
loading ../../data/VirusTotalnew/20180423/vt-20180423-0.csv
loading ../../data/VirusTotalnew/20180423/vt-20180423-2000.csv
loading ../../data/VirusTotalnew/20180412/vt-20180412-0.csv
loading ../../data/VirusTotalnew/20180412/vt-20180412-2000.csv
loading ../..

ERROR:root:File ../../data/VirusTotalnew/20180411/vt-20180411-2231.csv does not exist


loading ../../data/VirusTotalnew/20180421/vt-20180421-0.csv
loading ../../data/VirusTotalnew/20180421/vt-20180421-2000.csv
loading ../../data/VirusTotalnew/20180421/vt-20180421-4000.csv


In [14]:
df = df.drop_duplicates("url")

In [3]:
df = pd.read_csv("/home/sparrow/sparrow/data/VirusTotalnew/malicious/20180424.csv")

/home/sparrow/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df = df.loc[df.positives>0]

In [14]:
df.columns

Index([u'Unnamed: 0', u'positives', u'timestamp', u'total', u'url'], dtype='object')

In [19]:
df.to_csv("/home/sparrow/sparrow/data/VirusTotalnew/malicious/20180424-ma.csv", index = False)

In [37]:
df = pd.read_csv("/home/sparrow/sparrow/data/VirusTotalnew/malicious/20180424-ma.csv")

/home/sparrow/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
df = df.loc[df.positives>3]

In [21]:
df = df.set_index("url")

In [32]:
for i in list(df.index):
    try:
        if "asset.qingshu520.com" in i:
            print i
    except:
        print i

nan


In [33]:
df = pd.read_csv("/home/sparrow/sparrow/data/sangfor/malware_all_0419.csv")

/home/sparrow/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
for i in list(df.url):
    try:
        if "tu.dxcnd.cn" in i:
            print i
    except:
        print i

http://tu.dxcnd.cn/ggjc/update/20150331/fe1daa45fad2eade4f3b39194119d91d-2.db?auth_ver=2&appkey=4665fd2c6b0922a551e8ae74&nonce=1523242843426&lc=77qHTv4VtmRiXYtd&s=607b7a0cb66b4568fc449d7d49471b58
http://tu.dxcnd.cn
http://tu.dxcnd.cn/ggjc/update/20170927/8d36eba69ae16e63ab7717d5a79350d1-201709270.db
http://tu.dxcnd.cn/ggjc/update/20180327/10693454a770239c97385ddf28d7c40b-1803279990.db?auth_ver=2&appkey=50c82132bb394901f151ad96&nonce=1523254631674&model=T7&signmd5=608291140&vendor=CUBE&locale=en_US&pkg=cn.opda.a.phonoalbumshoushou&tk=UX5tr5IpcPXpC3cGrlflYA%3D%3D&vn=9.10.0&h=1824&w=1200&v=3911&ntt=WIFI&ie=862823021013547&lc=rZkSaTZAYNuzw7gP&dpi=320&sdk=19&s=e7f19d7de6b41917d03756ba1d944826
http://tu.dxcnd.cn/ggjc/update/20180327/10693454a770239c97385ddf28d7c40b-1803279990.db?auth_ver=2&appkey=50c82132bb394901f151ad96&nonce=1523217619513&tk=1B8piqDgcb0mPX256cBdRQ%3D%3D&sdk=22&dpi=320&vn=8.2.4.1&locale=zh_CN&ie=861270038093727&op=46000&h=1184&w=720&lc=farFVQdu0Lc27V4I&ntt=WIFI&v=3044&signm